In [3]:
import pandas as pd
import numpy as np

In [5]:
x_train = pd.read_csv('../Xtrain.csv', index_col='Id')
y_train = pd.read_csv('../Ytrain.csv', index_col='Id')
# to encoding values
to_dict = lambda d : dict(zip(d, range(len(d))))
to_dict_reverse = lambda d : dict(enumerate(d))

In [ ]:
# check mapping relation for bank and naics
bank_naics_mapping = zip(x_train['Bank'], x_train['NAICS'])
repeat = {}
for b, n in bank_naics_mapping:
    if n not in repeat:
        repeat[n] = set()
    repeat[n].add(b)

In [6]:
x_train.columns

value_mapping = {}
for name in ['Name', 'City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'RevLineCr', 'LowDoc']:
    value_mapping[name] = to_dict(set(x_train[name]))

In [40]:
time_to_value = lambda t: pd.to_datetime(t).value/10**11
approval_date = x_train['ApprovalDate'].apply(time_to_value)

In [41]:
distur_data = x_train['DisbursementDate'].apply(time_to_value)

In [44]:
x_train.columns

Index(['Name', 'City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS',
       'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob',
       'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc',
       'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'GrAppv',
       'SBA_Appv'],
      dtype='object')

In [71]:
columns_remains = ['ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob','RetainedJob', 'FranchiseCode',
 'UrbanRural']
currency_cols = ['DisbursementGross', 'BalanceGross', 'GrAppv','SBA_Appv']
name_cols = ['Name', 'City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'RevLineCr', 'LowDoc']
date_cols = ['ApprovalDate', 'DisbursementDate']

In [67]:
to_num = lambda money : int(money.strip('$').replace(',', '').split('.')[0])

In [ ]:
pd.options.mode.chained_assignment = None
x_train_processed = x_train[columns_remains]
for name in currenty_cols:
    x_train_processed[name] = x_train[name].apply(to_num)
for name in date_cols:
    x_train_processed[name] = x_train[name].apply(time_to_value)
for name in name_cols:
    x_train_processed[name] = x_train[name].apply(lambda n: value_mapping[name].get(n))

x_train_processed.loc[:, 'ApprovalFY'] = x_train_processed['ApprovalFY'].apply(lambda x: int(x[:4]))

x_train_processed = x_train_processed.dropna()
x_train_processed.loc[:, 'NewExist'] = x_train_processed['NewExist'].apply(int)

Index(['ChargeOff'], dtype='object')

In [116]:
y_train_processed = y_train.iloc[x_train_processed.index]

In [117]:
# split training set and test set
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(x_train_processed, y_train_processed, test_size=0.2, random_state=0)


In [118]:
# x_train_processed
from sklearn import preprocessing
x_normalized = preprocessing.normalize(train_data, norm='l2')

In [125]:
train_label.values.ravel()

array([0, 1, 0, ..., 0, 1, 1])

In [127]:
# from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(x_normalized, train_label.values.ravel())

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [129]:
from sklearn.metrics import accuracy_score
accuracy_score(test_label.values, clf.predict(preprocessing.normalize(test_data, norm='l2')))

0.5311531153115312

In [133]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.02)
model.fit(x_normalized, train_label.values.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.02, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [134]:
accuracy_score(test_label.values, model.predict(preprocessing.normalize(test_data, norm='l2')))

0.8216821682168217

In [140]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(train_data)
model=xgb.XGBClassifier(random_state=2,learning_rate=0.02, n_estimators = 200)
model.fit(std.transform(train_data), train_label.values.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.02, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [141]:
accuracy_score(test_label.values, model.predict(std.transform(test_data)))

0.8738873887388738

In [198]:
selected_columns = ['ApprovalDate', 'Term', 'NoEmp', 'NewExist', 'LowDoc', 'DisbursementGross', 'BalanceGross', 'GrAppv',
       'SBA_Appv']
selected_columns = train_data.columns

In [207]:
std = StandardScaler()
std.fit(train_data[selected_columns])
model=xgb.XGBClassifier(random_state=1,learning_rate=0.03, n_estimators = 200)
model.fit(std.transform(train_data[selected_columns]), train_label.values.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.03, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [208]:
model.feature_importances_

array([0.0736198 , 0.2753127 , 0.01300989, 0.02705499, 0.00416747,
       0.11688258, 0.01686723, 0.00847681, 0.03843874, 0.01634174,
       0.14778352, 0.05543707, 0.04836015, 0.01990944, 0.02842893,
       0.06186171, 0.00425807, 0.03741593, 0.00637329], dtype=float32)

In [206]:
selected_columns = [train_data.columns[i] for i in range(len(model.feature_importances_)) if model.feature_importances_[i] != 0]
selected_columns

['ApprovalFY',
 'Term',
 'NoEmp',
 'NewExist',
 'CreateJob',
 'FranchiseCode',
 'UrbanRural',
 'DisbursementGross',
 'GrAppv',
 'SBA_Appv',
 'ApprovalDate',
 'DisbursementDate',
 'State',
 'Zip',
 'Bank',
 'BankState',
 'NAICS',
 'RevLineCr',
 'LowDoc']

In [209]:
accuracy_score(test_label.values, model.predict(std.transform(test_data[selected_columns])))

0.8834883488348835